In [400]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_binary
import pandas as pd
import pprint
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.expected_conditions import visibility_of



In [372]:
years = ['2017','2016','2015','2014','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002','2001','1999']
print(years)
years.reverse()
print(years)

['2017', '2016', '2015', '2014', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '1999']
['1999', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2014', '2015', '2016', '2017']


In [402]:
years = ['2017','2016','2015','2014','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003']
years.reverse()
berlin_url = 'https://www.bmw-berlin-marathon.com/en/impressions/statistics-and-history/results-archive/'
driver = webdriver.Chrome()
driver.get(berlin_url)
#accept cookies
time.sleep(5)
driver.find_element_by_xpath('//button[@data-cookiefirst-action="accept"]').click()
time.sleep(1)
for year in years:
    #get specific year
    driver.find_element_by_xpath('//select[@class="events border-input"]').click()
    time.sleep(1)
    driver.find_element_by_xpath(f"//option[contains(text(),{year})]").click()
    time.sleep(2)
    master_results = []
    #get number of pages
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pages = soup.find_all('a',{"class": "page-link"})
    pages_text = [res.getText(',') for res in pages]
    #open all the splits
    for i in range(int(pages_text[7])-2):
        time.sleep(1)
        downloads = driver.find_element_by_xpath("//th[contains(text(), 'Place')]")
        actions = ActionChains(driver)
        actions.move_to_element(downloads).perform()
        time.sleep(1)
        btns = driver.find_elements(By.XPATH, f'//td[@class= " details-control"]')
        for button in btns:
            visibility_of(button)
            button.click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #scrape current page
        results = soup.find_all('tr')
        results_text = [res.getText(',') for res in results]
        accum = 0
        for i in range(21):
            master_results.append([results_text[accum] + ',' + results_text[accum+1]])
            accum += 2

        time.sleep(.2)

        #next page
        
        next = driver.find_element_by_xpath("//*[contains(text(), 'Next')]")
        actions = ActionChains(driver)
        actions.move_to_element(next).perform()
        next.click()
    #format data
    master_results_flat = [x for x in master_results]
    for i in range(len(master_results_flat)):
        master_results_flat[i] = master_results_flat[i][0].split(',')

    for i in range(20):
        for i in master_results_flat:
            if len(i) == 12:
                master_results_flat.remove(i)
            elif len(i) == 28:
                del i[5]
            elif len(i) != 27:
                master_results_flat.remove(i)

    #save to dataframe and csv
    df = pd.DataFrame(master_results_flat, columns = ['Overall_Place', 'First_Name','Last_Name', 'Country','SN', 'Sex','Net_time','Gross_time','split_times', '5k', '5k_split','10k', '10k_split','15k', '15k_split','20k', '20k_split','HM', 'HM_split','25k', '25k_split','30k', '30k_split','35k', '35k_split','40k', '40k_split'])
    
    df.to_csv(f'data/berlin_scrape_results/berlin_{year}.csv', header=True, index=False)
#print(master_results)


time.sleep(5)
driver.close()
driver.quit()

IndexError: list index out of range

In [324]:
int(pages_text[7])-1

1154

In [387]:
master_results_flat = [x for x in master_results]
for i in range(len(master_results_flat)):
    master_results_flat[i] = master_results_flat[i][0].split(',')
master_results_flat

[['Place',
  'First name',
  'Last name',
  'Nation',
  '#SN',
  'Club',
  'Sex',
  'Net time',
  'Clock time',
  'Photo',
  'Certificate',
  ''],
 ['1',
  'Joseph',
  'Ngolepus',
  'KEN',
  '91',
  'Kenia',
  'M',
  '02:08:47',
  '02:08:47',
  'Split times',
  '5km',
  '00:15:33',
  '10km',
  '00:30:50',
  '15km',
  '00:45:52',
  '20km',
  '01:00:58',
  'HM',
  '01:04:22',
  '25km',
  '01:16:08',
  '30km',
  '01:31:21',
  '35km',
  '01:46:19',
  '40km',
  '02:01:47'],
 ['2',
  'Willy',
  'Cheruiyot',
  'KEN',
  '6',
  'Kenia',
  'M',
  '02:09:08',
  '02:09:08',
  'Split times',
  '5km',
  '00:15:32',
  '10km',
  '00:30:50',
  '15km',
  '00:45:50',
  '20km',
  '01:01:03',
  'HM',
  '01:04:27',
  '25km',
  '01:16:07',
  '30km',
  '01:31:19',
  '35km',
  '01:46:17',
  '40km',
  '02:01:49'],
 ['3',
  'William',
  'Kiplagat',
  'KEN',
  '2',
  'Kenia',
  'M',
  '02:09:55',
  '02:09:55',
  'Split times',
  '5km',
  '00:15:24',
  '10km',
  '00:30:49',
  '15km',
  '00:45:49',
  '20km',
  '01:

In [391]:
for i in master_results_flat:
    if len(i) == 12:
        master_results_flat.remove(i)
    elif len(i) == 28:
        del i[5]
    elif len(i) != 27:
        master_results_flat.remove(i)
master_results_flat[1]

['2',
 'Willy',
 'Cheruiyot',
 'KEN',
 '6',
 'M',
 '02:09:08',
 '02:09:08',
 'Split times',
 '5km',
 '00:15:32',
 '10km',
 '00:30:50',
 '15km',
 '00:45:50',
 '20km',
 '01:01:03',
 'HM',
 '01:04:27',
 '25km',
 '01:16:07',
 '30km',
 '01:31:19',
 '35km',
 '01:46:17',
 '40km',
 '02:01:49']

In [392]:
df = pd.DataFrame(master_results_flat, columns = ['Overall_Place', 'First_Name','Last_Name', 'Country','SN', 'Sex','Net_time','Gross_time','split_times', '5k', '5k_split','10k', '10k_split','15k', '15k_split','20k', '20k_split','HM', 'HM_split','25k', '25k_split','30k', '30k_split','35k', '35k_split','40k', '40k_split'])


In [343]:
df.tail()

,Overall_Place,First_Name,Last_Name,Country,SN,Sex,Net_time,Gross_time,split_times,5k,...,HM,HM_split,25k,25k_split,30k,30k_split,35k,35k_split,40k,40k_split
43700,30730,Joseph,Emas,CAN,18404,M,06:57:29,07:12:47,Split times,5km,...,HM,03:03:52,25km,03:45:18,30km,04:42:41,35km,05:42:30,40km,06:35:21
43701,30731,Hans-Jürgen,Stampe,GER,3745,M,06:57:57,07:53:00,Split times,5km,...,HM,03:14:30,25km,03:55:22,30km,04:48:46,35km,05:41:57,40km,06:35:57
43702,30732,Dietmar,Schultejans,GER,36417,M,06:58:40,06:59:58,Split times,5km,...,HM,03:12:59,25km,03:51:51,30km,04:43:18,35km,05:38:42,40km,06:36:32
43703,30733,Bruno,Cavallini,ITA,15844,M,06:58:43,07:35:17,Split times,5km,...,HM,02:40:08,25km,03:24:20,30km,04:32:14,35km,05:33:00,40km,06:33:43
43704,13238,Maki,Murao,JPN,63886,W,07:02:03,07:13:25,Split times,5km,...,HM,02:57:39,25km,03:47:17,30km,04:46:03,35km,05:42:29,40km,06:39:10


In [393]:
df.to_csv('data/berlin_scrape_results/berlin_2001_to_page_571.csv', header=True, index=False)